In [2]:
using Revise
using Plots, CSV, DataFrames
includet("draftpolicy.jl")

┌ Info: Recompiling stale cache file /Users/evanmunro/.julia/compiled/v1.1/JuMP/DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /Users/evanmunro/.julia/compiled/v1.1/Ipopt/yMQMo.ji for Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1184


### I.  Example Draft Policy Plot

In [3]:
16+21+23

60

In [2]:
N=3
G= 20
s = Season(N,G)
size(s.matches)

(60, 2)

In [242]:
N=3
G= 20
s = Season(N,G)
abilities = [0,1/3,2/3]
record,incentives = calculate_draft_rule(s,nothing,true,abilities,10) 
summarize(record)

Final Draft probabilities: 
[0.643833, 0.343833, 0.0123333]
Stopped at match 37
Final record: 
[16, 21, 23]
Ability estimates: 
[0.217582, 0.362241, 0.420176]


In [264]:
0.644+0.0123/2

0.65015

In [253]:
plot(1:length(incentives),incentives,label=[""],xlabel="Match",ylabel="Bound on Draft Probability Change")
savefig("sim3_inc.pdf")

In [255]:
plot(1:(size(record.draft_prob,2)),transpose(record.draft_prob),xlabel="Match",ylabel="Percentage",
    color=[:blue :red :green], legend=:topright,label=["T1","T2","T3"]) 
savefig("sim3_prod.pdf")

In [256]:
plot(1:(size(record.cwins,2)),transpose(record.cwins),xlabel="Match",ylabel="Wins",
    color=[:blue :red :green], legend=:top,label=["T1","T2","T3"]) 
savefig("sim3_wins.pdf")

In [257]:
plot(1:(size(record.abilities,2)),transpose(record.abilities),xlabel="Match",ylabel="Abilities",
    color=[:blue :red :green], legend=:top,label=["T1","T2","T3"]) 
#plot(1:size(record.cwins,1),record.cwins,linestyle=:dash,color=[:blue :red :green], legend=:top,
  #  title="Win Records", 
   # label=["T1","T2","T3"],xlabel="Match",ylabel="Cumulative Wins")
savefig("sim3_abilities.pdf")




#



### II. Expected Gain over Uniform Lottery

In [265]:
print(length(s.matches[:,2]))

9

In [260]:
N=3
G=3
s = Season(N,G)

stops = zeros(20)
obj = zeros(20)

for m in 1:20
    o,st = expected_objective(s,[1/3,1/3,1/3],false,m)
    obj[m] = o
    stops[m] = st 
    println("done ", m)
end 

done 1
done 2
done 3
done 4
done 5
done 6
done 7
done 8
done 9
done 10
done 11
done 12
done 13
done 14
done 15
done 16
done 17
done 18
done 19
done 20


In [262]:
plot(1:20,obj.-0.5,color=[:blue],title="Expected Gain over Uniform Lottery",xlabel="B",ylabel="Probability",label=[""])
savefig("sim_obj.pdf")
plot(1:20,stops/size(s.matches,1),color=[:red],title="Games Taken Into Account",label=[""],xlabel="B",ylabel="Percent")
savefig("sim_stops.pdf")

## 2. Plotting Draft Policy on Real Data

In [106]:
#1985,1987,1988,1989
year = 1985 
team_id = CSV.read(string("/Users/evanmunro/Documents/Github/draft-policy/data/cleaned/teams_",
                            string(year),".csv"))
season = CSV.read(string("/Users/evanmunro/Documents/Github/draft-policy/data/cleaned/season_",year,".csv")) 
team_id

,team,id
,String⍰,Int64⍰
1,Atlanta Hawks,1
2,Boston Celtics,2
3,Chicago Bulls,3
4,Cleveland Cavaliers,4
5,Dallas Mavericks,5
6,Denver Nuggets,6
7,Detroit Pistons,7
8,Golden State Warriors,8
9,Houston Rockets,9


In [119]:
matches = convert(Matrix, season[[:hID,:aID]])
wins = convert(Array, season[:win])
teams = convert(Array,team_id[:id])
s = Season(teams,matches)
#0.5 had good results 
#wins
record=calculate_draft_rule(s,wins,1.05,16)
stoptime=record.stop
println(record.stop)
end_wins = record.cwins[:,size(record.cwins,2)]
end_draft = record.draft_prob[:,size(record.draft_prob,2)] 

df = DataFrame(team = copy(team_id[:team]),wins = copy(end_wins),score = copy(end_draft))
sort!(df, (:wins), rev=(true))

added_amt = sum(df[1:16,:score])/(length(teams)-16)
df[17:length(teams),:score] = df[17:length(teams),:score] .+ added_amt 
df[1:16,:score] =  0 
df[17:length(teams),:]
# reduce at 69 when multiple=5

1.05
16
97


,team,wins,score
,String⍰,Int64,Float64
1,Atlanta Hawks,34,0.131667
2,Kansas City Kings,31,0.15875
3,Los Angeles Clippers,31,0.111583
4,Seattle SuperSonics,31,0.136583
5,New York Knicks,24,0.1435
6,Golden State Warriors,22,0.12825
7,Indiana Pacers,22,0.189667


In [126]:
bottom3 = [16,8,10]
prob = record.draft_prob[bottom3,:]
wins = record.cwins[bottom3,:]
plot(transpose(prob[:,1:250]).+added_amt,label=["Knicks","Warriors","Spurs"],
    size=(800,250),
    #ylims=(0.05,0.3),
    linewidth=4,legend=:topright,xlabel="Game",title="Evolution of Draft Probability")
savefig("prob85.pdf")

In [121]:
plot(transpose(wins[:,1:250]),label=["Knicks","Warriors","Spurs","Stopping Time"],
    size=(800,250),
    linewidth=4,legend=:topleft,title="Cumulative Wins")
vline!([stoptime],color="black",label=["Knicks","Warriors","Spurs","Stopping Time"])
savefig("wins85.pdf")

In [94]:
year = 1989
team_id = CSV.read(string("/Users/evanmunro/Documents/Github/draft-policy/data/cleaned/teams_",
                            string(year),".csv"))
season = CSV.read(string("/Users/evanmunro/Documents/Github/draft-policy/data/cleaned/season_",year,".csv")) 
team_id

,team,id
,String⍰,Int64⍰
1,Atlanta Hawks,1
2,Boston Celtics,2
3,Charlotte Hornets,3
4,Chicago Bulls,4
5,Cleveland Cavaliers,5
6,Dallas Mavericks,6
7,Denver Nuggets,7
8,Detroit Pistons,8
9,Golden State Warriors,9


In [99]:
matches = convert(Matrix, season[[:hID,:aID]])
wins = convert(Array, season[:win])
teams = convert(Array,team_id[:id])
s = Season(teams,matches)
#0.5 had good results 
#wins
record=calculate_draft_rule(s,wins,15,16)
println(record.stop)
end_wins = record.cwins[:,size(record.cwins,2)]
end_draft = record.draft_prob[:,size(record.draft_prob,2)] 
df = DataFrame(team = copy(team_id[:team]),wins = copy(end_wins),score = copy(end_draft))
sort!(df, (:wins), rev=(true))

added_amt = sum(df[1:16,:score])/(length(teams)-16)
df[17:length(teams),:score] = df[17:length(teams),:score] .+ added_amt 
df[1:16,:score] =  0 
df[17:length(teams),:]

15
16
333


,team,wins,score
,String⍰,Int64,Float64
1,Portland Trail Blazers,39,0.00868519
2,Dallas Mavericks,38,0.00868519
3,Indiana Pacers,28,0.216352
4,Sacramento Kings,27,0.118019
5,New Jersey Nets,26,0.0136852
6,Los Angeles Clippers,21,0.0323519
7,San Antonio Spurs,21,0.0883519
8,Charlotte Hornets,20,0.0788519
9,Miami Heat,15,0.435019


In [104]:
bottom3 = [14,3,22]
prob = record.draft_prob[bottom3,:]
wins = record.cwins[bottom3,:]
plot(transpose(prob[:,1:500]).+added_amt,label=["Heat","Hornets","Spurs"],
    size=(800,250), linewidth=4,
    legend=:topleft,xlabel="Game",title="Evolution of Draft Probability")
savefig("prob89.pdf")

In [105]:
plot(transpose(wins[:,1:500]),label=["Heat","Hornets","Spurs","Stopping Time"],
    linewidth=4,size=(800,250),
    legend=:topleft,title="Cumulative Wins")
vline!([333],color="black",label=["Heat","Hornets","Spurs","Stopping Time"])
savefig("wins89.pdf")